###### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2023 Semester 1

## Assignment 1: Music genre classification with naive Bayes


**Student ID(s):**     1079860


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [ ]:
## RUN ALL CELLS IN ORDER FROM TOP TO BOTTOM AND THE CODE SHOULD WORK AS INTENDED ##


# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing
import sklearn

import random
import numpy as np
import pandas as pd

def preprocess(filename):
    df = pd.read_csv(filename)
           
    return df

In [ ]:
# This function should calculate prior probabilities and likelihood parameters from the training data and using them to build a naive Bayes model
def train(df):
	naive_bayes = {}
	naive_bayes['priors'] = calc_prior(df)
	naive_bayes['likelihood_params'] = calc_likelihood_params(df)
	return naive_bayes



# Calculates the prior probabilities for each class label
def calc_prior(df):
	prior_prob = {}
	labels = df.values[:, -1] # all rows, last column
	n = len(labels)
	unique_labels, counts = np.unique(labels, return_counts=True)
	#print(unique_labels)

	for i in range(len(unique_labels)):
		prior_prob[unique_labels[i]] = (counts[i] / n)

	return prior_prob


# Calculates the mean and variance of each class x label combination which define a gaussian distribution for them
def calc_likelihood_params(df):
	# Create an empty dictionary to store the likelihood parameters
	likelihood_params = {}
	unique_labels = np.unique(df.values[:, -1])
	features_list = df.columns[1:-1]
	
	for feature in features_list:

		# Create a nested dictionary for each feature
		likelihood_params[feature] = {}

		for label in unique_labels:
			# Create a nested dictionary for each label
			likelihood_params[feature][label] = {}

			# Create a list of feature values for this feature from the instances which match the current label
			feature_values = ((df[df['label'] == label])[feature])
			mean = 0
			
			# Calculate the mean for this feature x label combination
			for value in feature_values:
				mean += value / len(feature_values)
			#print(f"Mean of {feature},{label} is {mean:.4f}")

			sum_sqr_deviations = 0
			# Calculate the sum of squared deviations for this feature x label combination so we can calculate the variance
			for value in feature_values:
				sum_sqr_deviations += (value - mean)**2
			
			# Calculate the variance for this feature x label combination
			var = sum_sqr_deviations / (len(feature_values) - 1)
			#print(f"Variance of {feature},{label} is {var:.4f}")

			# Store the parameters in the dictionary
			likelihood_params[feature][label]['mean'] = mean
			likelihood_params[feature][label]['var'] = var

	return likelihood_params



In [ ]:
from numpy import inf

# This function should predict classes for new items in a test dataset
def predict(test_df, prior_prob, likelihood_params):
	argmax_labels = []

	for _, test_inst in test_df.iterrows():
		# Calculate posterior probabilities for each class for this test instance
		posterior_probs = calc_posterior(test_inst, prior_prob, likelihood_params)
		
		# Find argmax for this test instance
		#print("Now finding argmax for this instance")
		for post_prob in posterior_probs:
			max_prob = float(-inf)
			max_label = None
			for label in post_prob:
				#print(f"Current label: {label}")
				if label in prior_prob:
					prob = post_prob[label]
					#print(f"Current log-prob: {prob}")
					if prob > max_prob:
						max_prob = prob
						max_label = label
			if max_label is not None:
				argmax_labels.append(max_label)
			else:
				argmax_labels.append('none')
	return argmax_labels   


import math
log = math.log

# Computes the gaussian pdf for a given x value, mean and variance
def pdfnorm(x, mean, var):
	sqrt = math.sqrt
	PI = math.pi
	e = math.e
	
	return (1/(sqrt(2*PI*var))) * e ** -(((x-mean)**2)/(2*var))


# Calculates the log of the posterior probabilities for each class given a test instance
def calc_posterior(test_inst, prior_prob, likelihood_params):
	posterior_probs = []

	post_probs = {}
	for label in prior_prob:
		post_probs[label] = log(prior_prob[label])
		#print(f"Current label = {label}")
		for feature in likelihood_params:
			post_prob = pdfnorm(test_inst[feature], likelihood_params[feature][label]['mean'], likelihood_params[feature][label]['var'])
			#print(f"Current value = {test_inst[feature]}")
			#print(f"Current feature = {feature}")
			#print(f"Current mean = {likelihood_params[feature][label]['mean']}")
			#print(f"Current variance = {likelihood_params[feature][label]['var']}")
			#print(f"pdfnorm({feature}, {likelihood_params[feature][label]['mean']}, {likelihood_params[feature][label]['var']}) = {(pdfnorm(test_instance[feature], likelihood_params[feature][label]['mean'], likelihood_params[feature][label]['var']))}")
			#print(f"log(pdfnorm({feature}, {likelihood_params[feature][label]['mean']}, {likelihood_params[feature][label]['var']})) = {log(pdfnorm(test_instance[feature], likelihood_params[feature][label]['mean'], likelihood_params[feature][label]['var']))}")

			# cover the case where posterior probability is 0 to avoid domain error
			if post_prob != 0:
				post_probs[label] += log(post_prob)
			else:
				post_probs[label] = float(-inf)
				break
	posterior_probs.append(post_probs)

	return posterior_probs

In [ ]:
import sklearn.metrics as skm

# This function should evaluate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate(naive_bayes, test_df):
    evaluation = {}
    ground_truths = test_df['label']
    predictions = predict(test_df, naive_bayes['priors'], naive_bayes['likelihood_params'])

    #print(ground_truths)
    #print(predictions)

    # Store exhaustive evaluation metrics within a dictionary so they can be accessed as needed
    evaluation['report'] = skm.classification_report(ground_truths, predictions, output_dict=True)
    evaluation['accuracy'] = skm.accuracy_score(ground_truths, predictions)
    evaluation['precision'] = {}
    evaluation['precision']['macro'] = skm.precision_score(ground_truths, predictions, average="macro")
    evaluation['precision']['micro'] = skm.precision_score(ground_truths, predictions, average="micro")
    evaluation['precision']['weighted'] = skm.precision_score(ground_truths, predictions, average="weighted")
    evaluation['recall'] = {}
    evaluation['recall']['macro'] = skm.recall_score(ground_truths, predictions, average="macro")
    evaluation['recall']['micro'] = skm.recall_score(ground_truths, predictions, average="micro")
    evaluation['recall']['weighted'] = skm.recall_score(ground_truths, predictions, average="weighted")
    
    return evaluation

#evaluation = evaluate(train(preprocess("gztan_train.csv")), preprocess("gztan_test.csv"))
#print(evaluation['report'])

## Task 1. Pop vs. classical music classification

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.

### Q1
Compute and report the accuracy, precision, and recall of your model (treat "classical" as the "positive" class).

In [ ]:
import pandas as pd


evaluation = evaluate(train(preprocess("pop_vs_classical_train.csv")), preprocess("pop_vs_classical_test.csv"))
clasf_report = pd.DataFrame.from_dict(evaluation['report']).round(4)
display(clasf_report)
#pd.DataFrame.from_dict(evaluation['report'])

#classical precision = 95.2381%  => a few false positives
#classical recall = 100% => 0 false negatives => all true positives were captured
#accuracy = 97.6744%

#clasf_report.to_csv('task1q1.csv')

### Q2
For each of the features X below, plot the probability density functions P(X|Class = pop) and P(X|Class = classical). If you had to classify pop vs. classical music using just one of these three features, which feature would you use and why? Refer to your plots to support your answer.
- spectral centroid mean
- harmony mean
- tempo

In [ ]:
import matplotlib.pyplot as plt

train_df = preprocess("pop_vs_classical_train.csv")
likelihood_params = calc_likelihood_params(train_df)

feature_list = ['spectral_centroid_mean', 'harmony_mean', 'tempo']

labels = ['pop', 'classical']


# Create a list of x and y values to plot the pdf with using the range of the feature values
x_vals = {}
y_vals = {}
for label in labels:
    x_vals[label] = {}
    y_vals[label] = {}
    for feature in feature_list:
        x_min = min(train_df[feature])
        x_max = max(train_df[feature])
        x_inc = abs((x_min + x_max) / 100)
        feature_values = (train_df[train_df['label'] == label])[feature]
        x_vals[label][feature] = []
        y_vals[label][feature] = []
        i = x_min
        while (i < x_max):
            x_vals[label][feature].append(i)
            i += x_inc
        #x_vals[label][feature] = sorted(x_vals[label][feature])
        #print(x_vals[label][feature])
        y_vals[label][feature] = [pdfnorm(x, likelihood_params[feature][label]['mean'], likelihood_params[feature][label]['var']) for x in x_vals[label][feature]]


# Plot the pdfs
for feature in feature_list:
    plt.plot(x_vals['pop'][feature], y_vals['pop'][feature], label = "P(X|Class = Pop)")
    plt.plot(x_vals['classical'][feature], y_vals['classical'][feature], label = "P(X|Class = Classical)")
    plt.xlabel('feature value')
    plt.ylabel('pdf(x)')
    plt.title(f"{feature}")
    plt.legend()
    plt.show()


# If we had to classify pop or classical music using just one of these features then using the spectral centroid mean would be the best choice. This feature has the least amount of 
# overlap present between the probability density functions for both classes and therefore we can predict the class
# based on this attribute with a high degree of certainty. If the spectral centroid mean for an instance is below approximately 2000 then there is a larger probability that the instance 
# is an example of classical music and simulatenously a small probability that the instance is pop music, and vice versa if the spectral centroid mean is above 2000. Due to the lack of overlap between the PDs, 
# there is a very small chance of an error when a prediction is made purely using the probability density in relation to this single feature.
#  If the other 2 features were to be used to make predictions, predicting the class label
# based on which class has the higher probability density would likely result in more errors in our predictions due to the greater overlap between the probability densities for those attributes. (could provide example)

## Task 2. 10-way music genre classification

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.

### Q3
Compare the performance of the full model to a 0R baseline and a one-attribute baseline. The one-attribute baseline should be the best possible naive Bayes model which uses only a prior and a single attribute. In your write-up, explain how you implemented the 0R and one-attribute baselines.

### Q4
Train and test your model with a range of training set sizes by setting up your own train/test splits. With each split, use cross-fold validation so you can report the performance on the entire dataset (1000 items). You may use built-in functions to set up cross-validation splits. In your write-up, evaluate how model performance changes with training set size.

In [ ]:
complete_df = pd.concat([preprocess("gztan_train.csv"), preprocess("gztan_test.csv")])

#complete_df.head()
#complete_df.tail()

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

k_folds = [2,5,10]
evaluations = {}
concat_reports = {}
averaged_reports = {}

for k in k_folds:
	#print(k)
	evaluations[k] = []
	kf = StratifiedKFold(n_splits = k)#, shuffle = True, random_state = 2)
	
	# evaluate each training/test split combination generated with k folds
	for i, (train_index, test_index) in enumerate(kf.split(complete_df.values[:, 1:-1], complete_df.values[:,-1])):
		train_df = complete_df.iloc[train_index]
		test_df =  complete_df.iloc[test_index]

		evaluations[k].append(evaluate(train(train_df), test_df))
	reports = []
	for eval in evaluations[k]:
		reports.append(pd.DataFrame.from_dict(eval['report']))
	
	concat_reports[k] = pd.concat(reports)
	#display(concat_reports[k].index)	
	#display(concat_reports[k].columns)	
	#display(concat_reports[k])
	# average out evaluation metrics across the different training / test splits for the current k-folds
	averaged_reports[k] = ((concat_reports[k]).groupby(by=concat_reports[k].index, axis=0).mean())
	display(averaged_reports[k].round(3))
	#(averaged_reports[k].round(3)).to_csv(f"averaged-report-{k}-folds.csv")


### Q5
Implement a kernel density estimate (KDE) naive Bayes model and compare its performance to your Gaussian naive Bayes model. You may use built-in functions and automatic ("rule of thumb") bandwidth selectors to compute the KDE probabilities, but you should implement the naive Bayes logic yourself. You should give the parameters of the KDE implementation (namely, what bandwidth(s) you used and how they were chosen) in your write-up.

### Q6
Modify your naive Bayes model to handle missing attributes in the test data. Recall from lecture that you can handle missing attributes at test by skipping the missing attributes and computing the posterior probability from the non-missing attributes. Randomly delete some attributes from the provided test set to test how robust your model is to missing data. In your write-up, evaluate how your model's performance changes as the amount of missing data increases.